In [1]:
import numpy as np
from scipy.io import loadmat
import scipy as sp
import scipy.sparse.linalg as spalg
import math

In [2]:
# Read KOS Dataset
# Picked up this code from the LDA ipynb
with open('../data/docword.kos.txt', 'r') as df:
    num_docs = int(df.readline())
    num_words = int(df.readline())
    nnz = int(df.readline())

    X = sp.sparse.lil_matrix((num_docs, num_words))

    for l in df:
        d, w, v = [int(x) for x in l.split()]
        X[d-1, w-1] = v

# read KOS vocabulary
with open('../data/vocab.kos.txt', 'r') as vf:
    vocab = tuple(vf.read().split())

print("Vocabulary: {} words".format(len(vocab)))
print('Done reading KOS dataset.')


#Converting it to a floating point
#(Converted to CSC because the co-occurence part requires it in that format for the indices)
# Also made it a transpose 
#(This part can be made better later)
X = sp.sparse.csc_matrix(X.T.astype(np.float16))

#Note: X is now a words by doc matrix (6906 x 3430 for KOS)
print("Shape of X:" , X.shape)



Vocabulary: 6906 words
Done reading KOS dataset.
Shape of X: (6906, 3430)


In [ ]:
# Generating Word-Word Co-Occurence
numdocs = X.shape[1]
vocabSize = X.shape[0]
diag_X = np.zeros(vocabSize)
for i in range(X.indptr.size - 1):
    start = X.indptr[i]
    end = X.indptr[i + 1]

    Nm = np.sum(X.data[start:end])
    row_indices = X.indices[start:end]
    diag_X[row_indices] += X.data[start:end]/(Nm * (Nm - 1))
    X.data[start:end] = X.data[start:end]/math.sqrt(Nm * (Nm - 1))
    
C = X * X.transpose()/numdocs
C = C.todense()
C = np.array(C, copy = False)
diag_X = diag_X/numdocs
C = C - np.diag(diag_X)

print ('Test: Sum of C = ', np.sum(C))


Test: Sum of C =  0.999999994563


In [ ]:
# Generating Rectified Co-Occurence using Dykstra's Algorithm
"""
Generating the Positive Semi Definite Matrix Projection
"""
def proj_psd(Q,k):
    eigvals, eigvecs = spalg.eigs(Q)
    #print(eigvals.shape, eigvecs.shape)
    eigvals[eigvals.argsort()[:-k]] = 0
    # eigvecs * eigvals calculation
    MatPSD = np.einsum('ij,j->ij',eigvecs, eigvals)
    # (eigvecs * eigvals) * eigvecs' calculation
    MatPSD = sp.sparse.csc_matrix(MatPSD) * sp.sparse.csc_matrix(eigvecs.T)
    return MatPSD

"""
Generating the Normalized Matrix Projection
"""
def proj_norm(Q):
    #MatNorm = Q.todense()
    MatNorm = Q + ((1 - Q.sum())/(Q.shape[0] **2))
    return MatNorm

"""
Generating Non-Negative Matrix
"""
def proj_nn(Q):
    MatNN = Q
    MatNN[np.where(MatNN < 0)] = 0
    return MatNN

#Number of iterations
T = 10
#Total number of clusters
k = 10

P1 = np.zeros(C.shape)
P2 = np.zeros(C.shape)
P3 = np.zeros(C.shape)
X0 = C
for t in range(T):
    
    print("Iteration",t)
    
    #Projecting to Positive Semi Definite Matrix
    X0 = X0 + P1
    del P1
    X0 = sp.sparse.csc_matrix(X0)
    X1 = proj_psd(X0,k)
    P1 = X0 - X1
    del X0
    
    #Projecting to the Normalized Matrix
    X1 = X1 + P2
    del P2
    X2 = proj_norm(X1)
    P2 = X1 - X2
    del X1
    
    #Projecting to the Non Negative Matrix
    X2 = X2 + P3
    del P3
    X0 = proj_nn(X2)
    P3 = X2 - X0
    del X2
    
# Rectified Co-Occurence Matrix is C_star    
C_star = X0

print("Test for Non-Negativity: ", np.all(C_star >= 0))


Iteration 0
Iteration 1
